# Hyperparameter Tuning with Grid Search

This notebook demonstrates systematic hyperparameter optimization using grid search within nested cross-validation. This is essential for maximizing model performance while avoiding overfitting.

## Why Hyperparameter Tuning Matters

Default parameters rarely yield optimal performance. Tuning allows you to:
- **Adapt to your data**: Small samples need stronger regularization
- **Optimize for your metric**: Accuracy, AUC, or sensitivity may require different settings
- **Prevent overfitting**: Proper regularization is critical with high-dimensional neuroimaging data

## The Challenge: Data Leakage

**Problem**: If you optimize hyperparameters on the test set, you overfit the evaluation.

**Solution**: Nested cross-validation
- **Outer loop**: Evaluate final performance (unbiased)
- **Inner loop**: Optimize hyperparameters (can overfit, but that's OK)

## Approach Overview

**Feature Extraction**: Atlas-based ROIs  
**Feature Selection**: T-test with variable thresholds  
**Model**: Logistic Regression with tuned C and penalty  
**Validation**: Nested CV with GridSearch in inner loop

In [ ]:
import teslearn as tl
from teslearn.data import load_dataset_from_csv, NiftiLoader
from teslearn.features import AtlasFeatureExtractor
from teslearn.models import LogisticRegressionModel
from teslearn.selection import TTestSelector
from teslearn.cv import StratifiedKFoldValidator
from teslearn.pipeline import TESPipeline

from sklearn.model_selection import GridSearchCV
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## 1. Data Loading

Load your dataset as usual.

In [ ]:
# Load dataset
dataset = load_dataset_from_csv(
    csv_path='data/subjects.csv',
    target_col='response',
    task='classification'
)

loader = NiftiLoader()
images, indices = loader.load_dataset_images(dataset)
y = dataset.get_targets()

print(f"Dataset: {len(images)} subjects, {sum(y)} responders")

## 2. Define Hyperparameter Search Space

**Key parameters to tune**:

1. **Feature Selection Threshold**: How strict should the statistical test be?
   - Too loose: Many features, risk of overfitting
   - Too strict: Lose informative features

2. **Regularization Strength (C)**: Inverse of regularization
   - Small C: Strong regularization (good for small samples)
   - Large C: Weak regularization (good for large samples)

3. **Penalty Type**: L1 (sparse) vs L2 (dense) vs Elastic Net

**Best Practice**: Start with a coarse grid, then refine around promising regions.

In [ ]:
# Define hyperparameter grid
param_grid = {
    'model__C': [0.01, 0.1, 1.0, 10.0, 100.0],  # Regularization strength
    'model__penalty': ['l1', 'l2'],              # Lasso vs Ridge
}

print("Hyperparameter search space:")
print(f"  C values: {param_grid['model__C']}")
print(f"  Penalties: {param_grid['model__penalty']}")
print(f"  Total combinations: {len(param_grid['model__C']) * len(param_grid['model__penalty'])}")

# For feature selection threshold, we'll test a few values
p_thresholds = [0.001, 0.01, 0.05]
print(f"\nP-value thresholds to test: {p_thresholds}")

## 3. Feature Extraction

Extract features once - feature extraction doesn't have hyperparameters to tune.

In [ ]:
# Extract features
extractor = AtlasFeatureExtractor(
    atlas_path='data/atlas/HCP-MMP1.nii.gz',
    statistics=['mean', 'max', 'top10mean']
)

X = extractor.fit_transform(images)
print(f"Extracted {X.shape[1]} features")

## 4. Nested Cross-Validation with Grid Search

**Structure**:
- Outer loop: 5-fold CV for final evaluation
- Inner loop: 3-fold CV for hyperparameter tuning

**Important**: We fit the selector BEFORE grid search to avoid data leakage.

In [ ]:
from sklearn.metrics import roc_auc_score, accuracy_score

# Store results
results = []
best_params_per_fold = []

# Outer CV
outer_cv = StratifiedKFoldValidator(n_splits=5, shuffle=True, random_state=42)
inner_cv = StratifiedKFoldValidator(n_splits=3, shuffle=True, random_state=42)

fold_idx = 0

for train_idx, test_idx in outer_cv.split(X, y):
    fold_idx += 1
    print(f"\n{'='*50}")
    print(f"Fold {fold_idx}/5")
    print(f"{'='*50}")
    
    # Split data
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    images_train = [images[i] for i in train_idx]
    images_test = [images[i] for i in test_idx]
    
    # Test different p-value thresholds
    best_score = 0
    best_pipeline = None
    best_params = None
    
    for p_thresh in p_thresholds:
        # Feature selection (fit on training data only)
        selector = TTestSelector(p_threshold=p_thresh, correction=None)
        selector.fit(X_train, y_train)
        X_train_selected = selector.transform(X_train)
        
        # Skip if too few features selected
        if X_train_selected.shape[1] < 5:
            print(f"  p={p_thresh}: Too few features ({X_train_selected.shape[1]}), skipping")
            continue
        
        # Create pipeline
        model = LogisticRegressionModel(solver='lbfgs', max_iter=1000, random_state=42)
        pipeline = TESPipeline(
            feature_extractor=extractor,
            feature_selector=selector,
            model=model,
            use_scaling=True
        )
        
        # Grid search
        grid_search = GridSearchCV(
            pipeline,
            param_grid,
            cv=inner_cv,
            scoring='roc_auc',
            n_jobs=-1,
            verbose=0
        )
        
        # Fit grid search
        grid_search.fit(images_train, y_train)
        
        # Evaluate on test set
        y_pred_proba = grid_search.predict_proba(images_test)[:, 1]
        test_auc = roc_auc_score(y_test, y_pred_proba)
        
        print(f"  p={p_thresh}: Best params: {grid_search.best_params_}, Test AUC: {test_auc:.3f}")
        
        if test_auc > best_score:
            best_score = test_auc
            best_pipeline = grid_search.best_estimator_
            best_params = {
                'p_threshold': p_thresh,
                **grid_search.best_params_
            }
    
    # Store results for this fold
    y_pred = best_pipeline.predict(images_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    results.append({
        'fold': fold_idx,
        'accuracy': accuracy,
        'auc': best_score,
        'n_features': X_train_selected.shape[1] if 'X_train_selected' in locals() else 0,
    })
    best_params_per_fold.append(best_params)
    
    print(f"\nFold {fold_idx} Best: {best_params}")
    print(f"  Accuracy: {accuracy:.3f}, AUC: {best_score:.3f}")

## 5. Analyze Results

Look at:
1. Performance stability across folds
2. Most frequently selected hyperparameters
3. Relationship between regularization and performance

In [ ]:
# Convert to DataFrame
results_df = pd.DataFrame(results)

print("Cross-Validation Results Summary:")
print(f"  Mean Accuracy: {results_df['accuracy'].mean():.3f} (+/- {results_df['accuracy'].std():.3f})")
print(f"  Mean AUC: {results_df['auc'].mean():.3f} (+/- {results_df['auc'].std():.3f})")
print(f"\nPer-fold performance:")
print(results_df)

# Analyze selected hyperparameters
print("\nHyperparameter Selection Frequency:")
for param_name in ['model__C', 'model__penalty', 'p_threshold']:
    values = [p[param_name] for p in best_params_per_fold]
    print(f"\n{param_name}:")
    for val in set(values):
        count = values.count(val)
        print(f"  {val}: {count}/5 folds")

## 6. Visualize Hyperparameter Landscape

Understanding how different hyperparameters affect performance helps build intuition.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Performance across folds
axes[0, 0].bar(results_df['fold'], results_df['auc'], color='steelblue', alpha=0.7)
axes[0, 0].axhline(results_df['auc'].mean(), color='red', linestyle='--', 
                   label=f"Mean: {results_df['auc'].mean():.3f}")
axes[0, 0].set_xlabel('Fold')
axes[0, 0].set_ylabel('AUC')
axes[0, 0].set_title('Performance Across Folds')
axes[0, 0].legend()
axes[0, 0].set_ylim(0, 1)

# 2. C parameter distribution
c_values = [p['model__C'] for p in best_params_per_fold]
c_counts = pd.Series(c_values).value_counts().sort_index()
axes[0, 1].bar(range(len(c_counts)), c_counts.values, color='coral')
axes[0, 1].set_xticks(range(len(c_counts)))
axes[0, 1].set_xticklabels(c_counts.index)
axes[0, 1].set_xlabel('C Parameter')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].set_title('Selected C Values')

# 3. Penalty type distribution
penalty_values = [p['model__penalty'] for p in best_params_per_fold]
penalty_counts = pd.Series(penalty_values).value_counts()
axes[1, 0].pie(penalty_counts.values, labels=penalty_counts.index, autopct='%1.0f%%',
               colors=['lightblue', 'lightcoral'])
axes[1, 0].set_title('Penalty Type Distribution')

# 4. P-threshold distribution
p_values = [p['p_threshold'] for p in best_params_per_fold]
p_counts = pd.Series(p_values).value_counts().sort_index()
axes[1, 1].bar(range(len(p_counts)), p_counts.values, color='lightgreen')
axes[1, 1].set_xticks(range(len(p_counts)))
axes[1, 1].set_xticklabels([f"{p:.3f}" for p in p_counts.index])
axes[1, 1].set_xlabel('P-value Threshold')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].set_title('Selected P-value Thresholds')

plt.tight_layout()
plt.show()

## 7. Final Model Training

After hyperparameter tuning, train the final model on all data using the most common hyperparameters.

In [ ]:
from collections import Counter

# Select most common hyperparameters
final_c = Counter([p['model__C'] for p in best_params_per_fold]).most_common(1)[0][0]
final_penalty = Counter([p['model__penalty'] for p in best_params_per_fold]).most_common(1)[0][0]
final_p = Counter([p['p_threshold'] for p in best_params_per_fold]).most_common(1)[0][0]

print("Final Model Hyperparameters (majority vote):")
print(f"  C: {final_c}")
print(f"  Penalty: {final_penalty}")
print(f"  P-threshold: {final_p}")

# Train final pipeline
final_selector = TTestSelector(p_threshold=final_p, correction=None)
final_model = LogisticRegressionModel(
    C=final_c,
    penalty=final_penalty,
    solver='lbfgs' if final_penalty == 'l2' else 'liblinear',
    max_iter=1000,
    random_state=42
)

final_pipeline = TESPipeline(
    feature_extractor=extractor,
    feature_selector=final_selector,
    model=final_model,
    use_scaling=True
)

# Fit on all data
final_pipeline.fit(images, y)

print(f"\nFinal model trained on {len(images)} subjects")
print(f"Selected {len(final_pipeline.feature_importance_)} features")

## Key Takeaways

1. **Always use nested CV**: Prevents hyperparameter overfitting
2. **Grid search is expensive**: Consider RandomizedSearchCV for larger spaces
3. **Cross-validation stability**: Look at variance across folds, not just mean
4. **Majority vote**: Most common hyperparameters often work well

## Hyperparameter Tuning Checklist

- [ ] Define parameter grid (coarse → fine)
- [ ] Use appropriate inner CV (3-5 fold)
- [ ] Select scoring metric aligned with goal (AUC for imbalanced, accuracy for balanced)
- [ ] Analyze stability across outer folds
- [ ] Consider computational cost (fewer folds if needed)
- [ ] Validate on held-out test set (if available)

## Advanced Techniques

1. **Bayesian Optimization**: More efficient than grid search (use `scikit-optimize`)
2. **Randomized Search**: Sample from distributions instead of grid
3. **Early Stopping**: Stop if performance plateaus
4. **Nested permutation tests**: Validate that tuned model beats chance